<p align = "center" draggable=”false” ><img src="https://user-images.githubusercontent.com/37101144/161836199-fdb0219d-0361-4988-bf26-48b0fad160a3.png" 
     width="200px"
     height="auto"/>
</p>

# <h1 align="center" id="heading">Subscription Prediction with Delta Lake, PySpark, and MLlib</h1>

# Spark Environment

Make sure that you open this notebook in your Spark environment!

In [1]:
# !pip install -U -q pyspark delta-spark # If you use Colab uncomment this line

## Learning Objectives

At the end of this session, you will be able to 

- Load, save, partition data with Delta Lake tables
- Explore data with Spark DataFrames 
- Build a pipeline in MLlib for machine learning workflow
- Fit a logistic regression model, make predictions, and evaluate the model

## Part 1: Data Loader

We are using a dataset from the UCI Machine Learning Repository.

1. Use `wget` to download the dataset. Then use `ls` to verify that the `bank.zip` file is downloaded.

In [2]:
%%sh
wget https://archive.ics.uci.edu/ml/machine-learning-databases/00222/bank.zip

--2023-01-14 17:58:24--  https://archive.ics.uci.edu/ml/machine-learning-databases/00222/bank.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 579043 (565K) [application/x-httpd-php]
Saving to: ‘bank.zip’

     0K .......... .......... .......... .......... ..........  8%  239K 2s
    50K .......... .......... .......... .......... .......... 17%  966K 1s
   100K .......... .......... .......... .......... .......... 26% 3.41M 1s
   150K .......... .......... .......... .......... .......... 35%  719K 1s
   200K .......... .......... .......... .......... .......... 44% 3.28M 0s
   250K .......... .......... .......... .......... .......... 53% 5.10M 0s
   300K .......... .......... .......... .......... .......... 61% 2.85M 0s
   350K .......... .......... .......... .......... .......... 70% 1.47M 0s
   400K ........

In [3]:
ls ../dat

bank.zip


2. Unzip the file and use `ls` to see the files.

In [4]:
%%sh
unzip ../dat/bank.zip

Archive:  ../dat/bank.zip


replace bank-full.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename:  NULL
(EOF or read error, treating as "[N]one" ...)


CalledProcessError: Command 'b'unzip ../dat/bank.zip\n'' returned non-zero exit status 1.

In [ ]:
ls -lh ../dat

total 1136
-rw-r--r--  1 deshantm  staff   565K Jan 14 15:57 bank.zip


3. Inspect the dataset and note if there is anything to be aware of in the dataset structure. 

In [ ]:
%%sh
wc -l bank.csv

    4522 bank.csv


In [ ]:
%%sh
head bank.csv

"age";"job";"marital";"education";"default";"balance";"housing";"loan";"contact";"day";"month";"duration";"campaign";"pdays";"previous";"poutcome";"y"
30;"unemployed";"married";"primary";"no";1787;"no";"no";"cellular";19;"oct";79;1;-1;0;"unknown";"no"
33;"services";"married";"secondary";"no";4789;"yes";"yes";"cellular";11;"may";220;1;339;4;"failure";"no"
35;"management";"single";"tertiary";"no";1350;"yes";"no";"cellular";16;"apr";185;1;330;1;"failure";"no"
30;"management";"married";"tertiary";"no";1476;"yes";"yes";"unknown";3;"jun";199;4;-1;0;"unknown";"no"
59;"blue-collar";"married";"secondary";"no";0;"yes";"no";"unknown";5;"may";226;1;-1;0;"unknown";"no"
35;"management";"single";"tertiary";"no";747;"no";"no";"cellular";23;"feb";141;2;176;3;"failure";"no"
36;"self-employed";"married";"tertiary";"no";307;"yes";"no";"cellular";14;"may";341;1;330;2;"other";"no"
39;"technician";"married";"secondary";"no";147;"yes";"no";"cellular";6;"may";151;2;-1;0;"unknown";"no"
41;"entrepreneur";"marrie

4. Create a Delta table for `bank.csv` (Why Delta Lake? Here's a read [here](https://medium.com/@databeans-blogs/delta-lake-the-data-engineers-missing-piece-part-1-ebab66a3f8c0?source) from a data engineer's perspective)

    We first set up a Python project `ml-bank`, configure the SparkSession with the `configure_spark_with_delta_pip()` utility function in Delta Lake:

In [6]:
import pyspark
from delta import *

builder = pyspark.sql.SparkSession.builder.appName("ml-bank") \
  .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
  .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

spark = configure_spark_with_delta_pip(builder).getOrCreate()

23/01/14 18:00:46 WARN Utils: Your hostname, MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 192.168.68.124 instead (on interface en0)
23/01/14 18:00:46 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
:: loading settings :: url = jar:file:/Users/deshantm/anaconda3/envs/py39_12/lib/python3.9/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/deshantm/.ivy2/cache
The jars for the packages stored in: /Users/deshantm/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-ed9b3e03-f273-4183-9472-7959894cd310;1.0
	confs: [default]
	found io.delta#delta-core_2.12;2.2.0 in central
	found io.delta#delta-storage;2.2.0 in central
	found org.antlr#antlr4-runtime;4.8 in central
:: resolution report :: resolve 979ms :: artifacts dl 34ms
	:: modules in use:
	io.delta#delta-core_2.12;2.2.0 from central in [default]
	io.delta#delta-storage;2.2.0 from central in [default]
	org.antlr#antlr4-runtime;4.8 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |   0 

23/01/14 18:00:50 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


5. We define output formats and paths, you need to complete code to load the data from its source (since the delimiter of the file is semicolon, use [`spark.read.csv`](https://spark.apache.org/docs/latest/sql-data-sources-csv.html) that provides more flexibility) and write the data to its target (hint: [Create a table](https://docs.databricks.com/delta/delta-batch.html)).


__NOTE__ Databricks requires all the paths to be absolute, not relative. To check the absolute path, run `pwd` bash command.

In [8]:
# Define the input and output formats and paths and the table name.
write_format = 'delta'
#load_path = 'file:/databricks/driver/bank.csv'
load_path = 'file:/Users/deshantm/fourthbrain/Todd-MLE-11/assignments/week-05-big-data/nb/bank.csv'
save_path = 'file:/Users/deshantm/fourthbrain/Todd-MLE-11/assignments/week-05-big-data'
table_name = 'default.bank4k'

# Load the data from its source into a dataframe.
#read format with ; as delimiter
df = spark.read.format('csv').option('header', 'true').option('delimiter', ';').load(load_path)
#df = spark.read.format('csv').option('header', 'true').load(load_path)
df.printSchema()

#run the following 
#ALTER TABLE <table_name> SET TBLPROPERTIES (
#   'delta.columnMapping.mode' = 'name',
#   'delta.minReaderVersion' = '2',
#   'delta.minWriterVersion' = '5')
#alter table bank4k set tblproperties ('delta.minReaderVersion' = '2', 'delta.minWriterVersion' = '5')
spark.sql("DROP TABLE IF EXISTS bank4k")
spark.sql("CREATE TABLE bank4k USING delta LOCATION '/Users/deshantm/fourthbrain/Todd-MLE-11/assignments/week-05-big-data/bank4k'")
spark.sql("ALTER TABLE bank4k SET TBLPROPERTIES ('delta.minReaderVersion' = '2', 'delta.minWriterVersion' = '5')")

# Create table with path using DataFrame's schema and write data to it
df.write.format(write_format).mode('overwrite').option("overwriteSchema", "true").save(table_name)
# Note if you are overwriting to specificy overwrite option


root
 |-- age: string (nullable = true)
 |-- job: string (nullable = true)
 |-- marital: string (nullable = true)
 |-- education: string (nullable = true)
 |-- default: string (nullable = true)
 |-- balance: string (nullable = true)
 |-- housing: string (nullable = true)
 |-- loan: string (nullable = true)
 |-- contact: string (nullable = true)
 |-- day: string (nullable = true)
 |-- month: string (nullable = true)
 |-- duration: string (nullable = true)
 |-- campaign: string (nullable = true)
 |-- pdays: string (nullable = true)
 |-- previous: string (nullable = true)
 |-- poutcome: string (nullable = true)
 |-- y: string (nullable = true)

23/01/14 18:02:03 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


6. Verify what we just created.

In [ ]:
!ls -lhR /Users/deshantm/fourthbrain/Todd-MLE-11/assignments/week-05-big-data/nb/default.bank4k
!ls -lhR /Users/deshantm/fourthbrain/Todd-MLE-11/assignments/week-05-big-data/bank4k

total 120
drwxr-xr-x  4 deshantm  staff   128B Jan 14 16:56 _delta_log
-rw-r--r--  1 deshantm  staff    58K Jan 14 16:56 part-00000-769286d3-51ee-4b14-8382-140c139d914c-c000.snappy.parquet

/Users/deshantm/fourthbrain/Todd-MLE-11/assignments/week-05-big-data/nb/default.bank4k/_delta_log:
total 8
-rw-r--r--  1 deshantm  staff   3.0K Jan 14 16:56 00000000000000000000.json
total 0
drwxr-xr-x  10 deshantm  staff   320B Jan 14 16:56 _delta_log

/Users/deshantm/fourthbrain/Todd-MLE-11/assignments/week-05-big-data/bank4k/_delta_log:
total 32
-rw-r--r--  1 deshantm  staff   1.9K Jan 14 16:50 00000000000000000000.json
-rw-r--r--  1 deshantm  staff   1.9K Jan 14 16:50 00000000000000000001.json
-rw-r--r--  1 deshantm  staff   1.9K Jan 14 16:53 00000000000000000002.json
-rw-r--r--  1 deshantm  staff   1.9K Jan 14 16:56 00000000000000000003.json


7. Partition data by `job` status. 

  To speed up queries that have predicates involving the partition columns, we should partition data. Often time, we partition by anonymized user id; here we demonstrate the idea with `job`.

__NOTE__ We saw previously that the data can be loaded in Delta lake as one table. Now to partition the data, we need to remove the existing directory, or load it into a different directoty. The goal is to showcase delta lake capabilities. 

In [ ]:
import shutil
shutil.rmtree('/Users/deshantm/fourthbrain/Todd-MLE-11/assignments/week-05-big-data/bank4k') # To replace data, we need to remove the existing directory

In [26]:
partition_by = 'job'

#write to target with partition by job
df.write.format(write_format).mode('overwrite').option("overwriteSchema", "true").partitionBy(partition_by).save(table_name)

23/01/14 18:34:56 ERROR Executor: Exception in task 0.0 in stage 62.0 (TID 586)
java.lang.AssertionError: assertion failed
	at scala.Predef$.assert(Predef.scala:208)
	at org.apache.spark.sql.execution.datasources.parquet.ParquetFileFormat.$anonfun$buildReaderWithPartitionValues$1(ParquetFileFormat.scala:261)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.org$apache$spark$sql$execution$datasources$FileScanRDD$$anon$$readCurrentFile(FileScanRDD.scala:209)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.nextIterator(FileScanRDD.scala:270)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.hasNext(FileScanRDD.scala:116)
	at org.apache.spark.sql.execution.FileSourceScanExec$$anon$1.hasNext(DataSourceScanExec.scala:554)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.columnartorow_nextBatch_0$(Unknown Source)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCo

Py4JJavaError: An error occurred while calling o236.save.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.jobAbortedError(QueryExecutionErrors.scala:651)
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.write(FileFormatWriter.scala:278)
	at org.apache.spark.sql.delta.files.TransactionalWrite.$anonfun$writeFiles$1(TransactionalWrite.scala:397)
	at org.apache.spark.sql.execution.SQLExecution$.$anonfun$withNewExecutionId$6(SQLExecution.scala:109)
	at org.apache.spark.sql.execution.SQLExecution$.withSQLConfPropagated(SQLExecution.scala:169)
	at org.apache.spark.sql.execution.SQLExecution$.$anonfun$withNewExecutionId$1(SQLExecution.scala:95)
	at org.apache.spark.sql.SparkSession.withActive(SparkSession.scala:779)
	at org.apache.spark.sql.execution.SQLExecution$.withNewExecutionId(SQLExecution.scala:64)
	at org.apache.spark.sql.delta.files.TransactionalWrite.writeFiles(TransactionalWrite.scala:352)
	at org.apache.spark.sql

In [ ]:
ls -lh /Users/deshantm/fourthbrain/Todd-MLE-11/assignments/week-05-big-data/nb/default.bank4k

total 120
drwxr-xr-x  6 deshantm  staff   192B Jan 14 17:11 _delta_log/
drwxr-xr-x  4 deshantm  staff   128B Jan 14 17:11 job=admin./
drwxr-xr-x  4 deshantm  staff   128B Jan 14 17:11 job=blue-collar/
drwxr-xr-x  4 deshantm  staff   128B Jan 14 17:11 job=entrepreneur/
drwxr-xr-x  4 deshantm  staff   128B Jan 14 17:11 job=housemaid/
drwxr-xr-x  4 deshantm  staff   128B Jan 14 17:11 job=management/
drwxr-xr-x  4 deshantm  staff   128B Jan 14 17:11 job=retired/
drwxr-xr-x  4 deshantm  staff   128B Jan 14 17:11 job=self-employed/
drwxr-xr-x  4 deshantm  staff   128B Jan 14 17:11 job=services/
drwxr-xr-x  4 deshantm  staff   128B Jan 14 17:11 job=student/
drwxr-xr-x  4 deshantm  staff   128B Jan 14 17:11 job=technician/
drwxr-xr-x  4 deshantm  staff   128B Jan 14 17:11 job=unemployed/
drwxr-xr-x  4 deshantm  staff   128B Jan 14 17:11 job=unknown/
-rw-r--r--  1 deshantm  staff    58K Jan 14 16:56 part-00000-769286d3-51ee-4b14-8382-140c139d914c-c000.snappy.parquet


  We only touch the surface of Delta Lake, for more information, check [Delta Lake guide](https://docs.databricks.com/delta/index.html)

## Part 2: Exploring The Data

We will use the direct marketing campaigns (phone calls) of a Portuguese banking institution. The classification goal is to predict whether the client will subscribe (Yes/No) to a term deposit.


1. Load data from its source by specifying the data format and path; then check out the schemas.

In [87]:
read_format = 'delta'
load_path = '/Users/deshantm/fourthbrain/Todd-MLE-11/assignments/week-05-big-data/nb/default.bank4k'

df = spark.read.format(read_format).load(load_path) 

df.printSchema()

root
 |-- age: string (nullable = true)
 |-- job: string (nullable = true)
 |-- marital: string (nullable = true)
 |-- education: string (nullable = true)
 |-- default: string (nullable = true)
 |-- balance: string (nullable = true)
 |-- housing: string (nullable = true)
 |-- loan: string (nullable = true)
 |-- contact: string (nullable = true)
 |-- day: string (nullable = true)
 |-- month: string (nullable = true)
 |-- duration: string (nullable = true)
 |-- campaign: string (nullable = true)
 |-- pdays: string (nullable = true)
 |-- previous: string (nullable = true)
 |-- poutcome: string (nullable = true)
 |-- y: string (nullable = true)



Here are the columns you should see:

* Input variables: age, job, marital, education, default, balance, housing, loan, contact, day, month, duration, campaign, pdays, previous, poutcome

* Output variable: y (deposit)

2. Have a peek of the first five observations. Use the `.show()` method.

In [43]:
#use show to peek at first five observations
df.show(5)

+---+----------+-------+---------+-------+-------+-------+----+--------+---+-----+--------+--------+-----+--------+--------+---+
|age|       job|marital|education|default|balance|housing|loan| contact|day|month|duration|campaign|pdays|previous|poutcome|  y|
+---+----------+-------+---------+-------+-------+-------+----+--------+---+-----+--------+--------+-----+--------+--------+---+
| 30|unemployed|married|  primary|     no|   1787|     no|  no|cellular| 19|  oct|      79|       1|   -1|       0| unknown| no|
| 40|unemployed|married|secondary|     no|    219|    yes|  no|cellular| 17|  nov|     204|       2|  196|       1| failure| no|
| 31|unemployed| single|  primary|     no|    406|     no|  no|cellular|  4|  feb|     736|       1|   -1|       0| unknown|yes|
| 56|unemployed|married|  primary|     no|   3391|     no|  no|cellular| 21|  apr|     243|       1|   -1|       0| unknown|yes|
| 24|unemployed| single|secondary|     no|    119|     no|  no| unknown| 15|  sep|      51|      

To get a prettier result, it can be nice to use Pandas to display our DataFrame. Use the Spark `.take()` method to get the first 5 rows and then convert to a pandas DataFrame. Don't forget to pass along the column names. You should see the same result as above, but in a more aesthetically appealing format.

In [51]:
import pandas as pd
df_take = pd.DataFrame(df.take(5), columns=df.columns)
df_take.head




<bound method NDFrame.head of   age         job  marital  education default balance housing loan   contact  \
0  30  unemployed  married    primary      no    1787      no   no  cellular   
1  40  unemployed  married  secondary      no     219     yes   no  cellular   
2  31  unemployed   single    primary      no     406      no   no  cellular   
3  56  unemployed  married    primary      no    3391      no   no  cellular   
4  24  unemployed   single  secondary      no     119      no   no   unknown   

  day month duration campaign pdays previous poutcome    y  
0  19   oct       79        1    -1        0  unknown   no  
1  17   nov      204        2   196        1  failure   no  
2   4   feb      736        1    -1        0  unknown  yes  
3  21   apr      243        1    -1        0  unknown  yes  
4  15   sep       51        1    -1        0  unknown   no  >

3. We can also perform transformations on our DataFrame using the Pandas commands that we know and love using the [Pandas on Spark API](https://spark.apache.org/docs/latest/api/python/user_guide/pandas_on_spark/index.html) **(new on Spark versions >= 3.2)**.  Pandas on Spark API was born out of the Databricks project, Koalas 🐨, allows us to use the Pandas commands and aesthetically pleasing output that we know and love distributed on the speed and scale of Spark!  For a nice quickstart on Pandas on Spark, check out this [article](https://towardsdatascience.com/run-pandas-as-fast-as-spark-f5eefe780c45)!  Let's convert our Spark DataFrame to a Pandas on Spark DataFrame!

In [53]:
import pyspark.pandas as ps

psdf = df.pandas_api()
psdf.head(5)

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,35,management,single,tertiary,no,1350,yes,no,cellular,16,apr,185,1,330,1,failure,no
1,30,management,married,tertiary,no,1476,yes,yes,unknown,3,jun,199,4,-1,0,unknown,no
2,35,management,single,tertiary,no,747,no,no,cellular,23,feb,141,2,176,3,failure,no
3,40,management,married,tertiary,no,194,no,yes,cellular,29,aug,189,2,-1,0,unknown,no
4,38,management,divorced,unknown,no,0,yes,no,cellular,18,nov,96,2,-1,0,unknown,no


4. How many datapoints are there in the dataset? Use the `.count()` method.

In [54]:
psdf.count()

/Users/deshantm/anaconda3/envs/py39_12/lib/python3.9/site-packages/pyspark/pandas/internal.py:1573: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  fields = [
/Users/deshantm/anaconda3/envs/py39_12/lib/python3.9/site-packages/pyspark/sql/pandas/conversion.py:486: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


age          4521
job          4521
marital      4521
education    4521
default      4521
balance      4521
housing      4521
loan         4521
contact      4521
day          4521
month        4521
duration     4521
campaign     4521
pdays        4521
previous     4521
poutcome     4521
y            4521
dtype: int64

5. Use the `.describe()` method to see summary statistics on the features.

In [55]:
psdf.describe()

/Users/deshantm/anaconda3/envs/py39_12/lib/python3.9/site-packages/pyspark/pandas/internal.py:1573: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  fields = [
/Users/deshantm/anaconda3/envs/py39_12/lib/python3.9/site-packages/pyspark/sql/pandas/conversion.py:486: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
count,4521,4521,4521,4521,4521,4521,4521,4521,4521,4521,4521,4521,4521,4521,4521,4521,4521
unique,67,12,3,4,2,2353,2,2,3,31,12,875,32,292,24,4,2
top,34,management,married,secondary,no,0,yes,no,cellular,20,may,123,1,-1,0,unknown,no
freq,231,969,2797,2306,4445,357,2559,3830,2896,257,1398,27,1734,3705,3705,3705,4000


6. The above result includes the columns that are categorical, so those columns don't have useful summary statistics. Let's inspect just the numeric features.

    `numeric_features` is defined below to contain the column names of the numeric features.  Notice we use the `zip` functions to iterate through two lists at the same time!
    
    Filter the DataFrame as you would in pandas to select only the numeric features from the DataFrame and then get the summary statistics on the resulting DataFrame as we did above.

In [112]:
col_names = [name for name in psdf.dtypes.index]
print(col_names)
#get a list of data types of the psdf columns with inferSchema = True
#list datatypes of each column by inferSchema = True
df = spark.read.format(read_format).options(inferSchema=True).load(load_path)

#print(psdf.dtypes.inferSchema).tolist()
dtypes = [dtype for dtype in psdf.dtypes]
print(dtypes)
col_names = [name for name in psdf.dtypes.index]
dtypes = [dtype for dtype in psdf.dtypes.tolist()]
print(dtypes)
numeric_features = [name for name, dtype in zip(col_names, dtypes) if dtype == 'int32' or dtype == 'float32' or dtype == 'float64']
#numeric_features = [name for name, dtype in zip(col_names, dtypes) if dtype == 'int32' or dtype == 'float32' or dtype == 'float64']

print(psdf[numeric_features].head(5))
print(psdf[numeric_features].count())

['age', 'job', 'marital', 'education', 'default', 'balance', 'housing', 'loan', 'contact', 'day', 'month', 'duration', 'campaign', 'pdays', 'previous', 'poutcome', 'y']
[dtype('O'), dtype('O'), dtype('O'), dtype('O'), dtype('O'), dtype('O'), dtype('O'), dtype('O'), dtype('O'), dtype('O'), dtype('O'), dtype('O'), dtype('O'), dtype('O'), dtype('O'), dtype('O'), dtype('O')]
[dtype('O'), dtype('O'), dtype('O'), dtype('O'), dtype('O'), dtype('O'), dtype('O'), dtype('O'), dtype('O'), dtype('O'), dtype('O'), dtype('O'), dtype('O'), dtype('O'), dtype('O'), dtype('O'), dtype('O')]
Empty DataFrame
Columns: []
Index: [0, 1, 2, 3, 4]


/Users/deshantm/anaconda3/envs/py39_12/lib/python3.9/site-packages/pyspark/pandas/series.py:404: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  s = pd.Series(
/Users/deshantm/anaconda3/envs/py39_12/lib/python3.9/site-packages/pyspark/pandas/internal.py:1573: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  fields = [
/Users/deshantm/anaconda3/envs/py39_12/lib/python3.9/site-packages/pyspark/sql/pandas/conversion.py:486: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


Series([], dtype: float64)


7. Run the following code to look at correlation between the numeric features.  Let's convert our Pandas on Spark DataFrame to a Pandas DataFrame using the `to_pandas()` command. This will enable us to plot the data.  What do you see?

In [ ]:
# Convert Pandas on Spark DataFrame to Spark DataFrame
numeric_data = psdf[numeric_features].to_pandas()

axs = pd.plotting.scatter_matrix(numeric_data, figsize=(8, 8));
n = len(numeric_data.columns)

for i in range(n):
    v = axs[i, 0]
    v.yaxis.label.set_rotation(0)
    v.yaxis.label.set_ha('right')
    v.set_yticks(())
    h = axs[n - 1, i]
    h.xaxis.label.set_rotation(90)
    h.set_xticks(())

There aren't any highly correlated variables, implying that we can keep them all for the model. However, day and month columns are not really useful, so will remove these two columns.

8. Use the `.drop()` method to remove the `month` and `day` columns.
    
    Note that this method returns a new DataFrame, so save that result as `sdf`.

    Use the `.dtypes` method to verify that `sdf` now has the correct columns.

In [ ]:
# [YOUR CODE HERE]

## Part 3: Preparing Data for Training a Model

What follows is something analagous to a dataloader pipeline in Tensorflow--we're going to chain together some transformations that will convert our categorical variables into a one-hot format more amenable to training a machine learning model. 
The next code cell just sets this all up, but it doesn't run these transformations on our data yet.

The process includes Category Indexing, One-Hot Encoding and VectorAssembler — a feature transformation that merges multiple columns into a vector column.

The code is taken from [databricks’ official site](https://docs.databricks.com/applications/machine-learning/train-model/mllib/index.html#binary-classification-example) and it indexes each categorical column using the StringIndexer, then converts the indexed categories into one-hot encoded variables. 
The resulting output has the binary vectors appended to the end of each row. 
We use the StringIndexer again to encode our labels to label indices. 
Next, we use the VectorAssembler to combine all the feature columns into a single vector column.

1. Complete the code by completing the assignment of `assembler`. Use `VectorAssembler` and pass in `assemblerInputs` as `inputCols` and name the `outputCol` `"features"`.

In [ ]:
from pyspark.ml.feature import OneHotEncoder , StringIndexer, VectorAssembler

categoricalColumns = ['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'poutcome']
stages = []

for categoricalCol in categoricalColumns:
    stringIndexer = StringIndexer(inputCol = categoricalCol, outputCol = categoricalCol + 'Index')
    encoder = OneHotEncoder(inputCols=[stringIndexer.getOutputCol()], outputCols=[categoricalCol + "classVec"])
    stages += [stringIndexer, encoder]

label_stringIdx = StringIndexer(inputCol = 'y', outputCol = 'label')
stages += [label_stringIdx]
numericCols = ['age', 'balance', 'duration', 'campaign', 'pdays', 'previous']
assemblerInputs = [c + "classVec" for c in categoricalColumns] + numericCols
assembler = # [YOUR CODE HERE]
stages += [assembler]

## Part 4: Pipeline

We use Pipeline to chain multiple transformations and estimators together to specify our machine learning workflow. 
A Pipeline’s stages are specified as an ordered array.  
To run the pipeline on our Pandas on Spark DataFrame, we will convert it back to a Spark DataFrame using the **`to_spark()`** command

1. Fit a pipeline on df.

In [ ]:
from pyspark.ml import Pipeline
sdf = sdf.to_spark()
pipeline = Pipeline(stages=stages)

pipelineModel = # [YOUR CODE HERE]

2. Transform `pipelineModel` on `df` and assign this to variable `transformed_df`.

In [ ]:
transformed_df = None # [YOUR CODE HERE]
transformed_df.printSchema()

From the transformation, we'd like to take the `label` and `features` columns as well as the original columns from `sdf.`

3. Use the `.select()` method to pull these columns from the `transformed_df` and reassign the resulting DataFrame to `sdf`.

In [ ]:
selectedCols = ['label', 'features'] + sdf.columns
sdf = None # [YOUR CODE HERE]
sdf.printSchema()

4. Let's view the first five rows of the `sdf` DataFrame using the methods we learned in Part 2:
    * `.show()` method
    * `.take()` method and convert result to a Pandas DataFrame

In [ ]:
# [YOUR CODE HERE]

5. Randomly split the dataset in training and test sets, with 70% of the data in the training set and the remaining 30% in the test set.

    Hint: Call the `.randomSplit()` method.

In [ ]:
train, test = None # [YOUR CODE HERE]

6. What are the sizes of the training and test sets?

In [ ]:
# [YOUR CODE HERE]

## Part 5: Logistic Regression Model

Optional:
- You can build a RandomForestClassifier with : from pyspark.ml.classification import RandomForestClassifier
- You can build a Gradient-Boosted Tree Classifier with : from pyspark.ml.classification import GBTClassifier

1. Fit a logistic regression with `featuresCol` as `"features"`, `labelCol` as `"label"` and a `maxIter` of 10.

In [ ]:
from pyspark.ml.classification import LogisticRegression

# [YOUR CODE HERE]

2. We can obtain the coefficients by using logistic regression model’s attributes. Look at the following plot of the beta coefficients.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
beta = np.sort(lrModel.coefficients)
plt.plot(beta)
plt.ylabel('Beta Coefficients')
plt.show()

3. Use the `.transform()` method to make predictions and save them as `predictions`.

In [ ]:
predictions = None # [YOUR CODE HERE]

4. View the first 10 rows of the `predictions` DataFrame.

In [ ]:
# [YOUR CODE HERE]

5. What is the area under the curve?

    You can find it with the `evaluator.evaluate()` function.

In [ ]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

evaluator = BinaryClassificationEvaluator()
# [YOUR CODE HERE]

## OPTIONAL: HyperParameter Tuning a Gradient-Boosted Tree Classifier

1. Fit and make predictions using `GBTClassifier`. The syntax will match what we did above with `LogisticRegression`.

In [ ]:
from pyspark.ml.classification import GBTClassifier

gbt = GBTClassifier(maxIter=10)
gbtModel = gbt.fit(train)
predictions = gbtModel.transform(test)
predictions.select('age', 'job', 'label', 'rawPrediction', 'prediction', 'probability').show(10)

2. Perform cross-validation to compare different parameters.

    Note that it can take a while because it's training over many gradient boosted trees. Give it at least 10 minutes to complete.

In [ ]:
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
paramGrid = (ParamGridBuilder()
             .addGrid(gbt.maxDepth, [2, 4, 6])
             .addGrid(gbt.maxBins, [20, 60])
             .addGrid(gbt.maxIter, [10, 20])
             .build())
cv = CrossValidator(estimator=gbt, estimatorParamMaps=paramGrid, evaluator=evaluator, numFolds=5)
cvModel = cv.fit(train)
predictions = cvModel.transform(test)
evaluator.evaluate(predictions)

## Acknowledgements


This notebook is adapted from [Machine Learning with PySpark and MLlib](https://towardsdatascience.com/machine-learning-with-pyspark-and-mllib-solving-a-binary-classification-problem-96396065d2aa)